In [ ]:
import os
import nltk
import re
import pandas as pd
import dateutil.parser


In [ ]:
base_path = '/home/Data/Text_Parsing_Project'
filename = 'summary27.txt'
path_to_file = os.path.join(base_path, filename)
fd = open(path_to_file , 'r')

with fd:
    text = fd.read()

In [ ]:
from nltk import sent_tokenize
sentences = sent_tokenize(text)


In [ ]:
#Regex expression to build the list of paragraphs in "Prevention" category:

pattern = '(?s)(?<=new, recovered and deceased cases.).*(?=Overview)'
prevention_all = re.findall(pattern, text)
list_prevention_paragraphs = re.findall('.*\n', prevention_all[0])
list_prevention_paragraphs = [e for e in list_prevention_paragraphs if e[0:2] == 'On' or e[0:4] == 'From']
list_govt_paragraphs = []
list_references = []


for p in list_prevention_paragraphs:
    if 'A' in p or 'B' in p or 'C' in p or 'D' in p:
        list_govt_paragraphs.append(p)
        list_prevention_paragraphs.remove(p)

#Bulding the list of 'Responsive Action' paragraphs from the Gender-based violence section:        

pattern = '(?s)(?<=gender-based violence\n).*(?=Shelter)'
dom_violence_all = re.findall(pattern, text)
dom_violence_paragraphs = re.findall('.*\n', dom_violence_all[0])

#Only relevant paragraphs are included, \n list entries are omitted:

list_responsive_action_paragraphs = [e for e in dom_violence_paragraphs if e[0:2] == 'On' or e[0:4] == 'From']
list_govt_paragraphs = [e for e in list_govt_paragraphs if e[0:2] == 'On' or e[0:4] == 'From']


for _ in range(len(list_prevention_paragraphs)):
    list_references.append('NRP pg. 59 Prevention')
for _ in range(len(list_govt_paragraphs)):
    list_references.append('Government measures')
for _ in range(len(list_responsive_action_paragraphs)):
    list_references.append('NRP pg. 49 Responsive actions')

list_all_paragraphs = (list_prevention_paragraphs) + (list_govt_paragraphs) + (list_responsive_action_paragraphs)

In [ ]:
list_all_ids = [num for num in range(1, len(list_all_paragraphs) + 1)]


In [ ]:
#Lists of dates for "Dates" column, to be matched with "Prevention" and "Domestic Violence" categories

pattern = re.compile(r'(?:(?<=On\s)|(?<=\d\sto\s)).*?(?=,)')

list_dates_prevention = re.findall(pattern, ' '.join(list_prevention_paragraphs))
list_dates_govt = re.findall(pattern, ' '.join(list_govt_paragraphs))
list_dates_responsive_action = re.findall(pattern, dom_violence_all[0])

list_all_dates = list_dates_prevention + list_dates_govt + list_dates_responsive_action

In [ ]:
#Create a list of locations for 'Geo. Area' from all_paragraphs here:

list_areas = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']


In [ ]:
#current source

pattern = '(?<=Report No: )\d{2}'
report_num = re.findall(pattern, text)[0]
source = f'Weekly COVID-19 Summary Report No: {report_num}'

In [ ]:
#Create the dictionary here

dict_all = {}
for i in range(len(list_all_ids)):
    dict_all[list_all_ids[i]] = {'Date': list_all_dates[i], 'Contributing unit': 'RC MI', 'Impact': 'Municipality level', 'Geo. Area': [], 'Ref. to National Response Plan and other government documents': list_references[i], 'Description': list_all_paragraphs[i], 'Sources/Remarks/Recommendations': source}
    for area in list_areas:
        if area in dict_all[list_all_ids[i]]['Description']:
            dict_all[list_all_ids[i]]['Geo. Area'].append(area)
    dict_all[list_all_ids[i]]['Geo. Area'] = ', '.join(dict_all[list_all_ids[i]]['Geo. Area'])
    dict_all[list_all_ids[i]]['Geo. Area'] = dict_all[list_all_ids[i]]['Geo. Area'].replace("'", "")
    

In [ ]:
#df_all = pd.DataFrame(dict_all)
#(dict_all['1'],columns=['Description'])
#df_all = pd.DataFrame.from_dict(dict_all)
#[list_all_ids],columns=['Date','Contributing unit','Impact','Geo. Area','Ref. to National Response Plan and other government documents','Description','Sources/Remarks/Recommendations']

df_all = pd.DataFrame.from_dict(dict_all, orient='index')

df_all['Date'] = df_all['Date'].apply(lambda x: dateutil.parser.parse(x).date())
df_all.to_excel('covid_weekly.xlsx', index=False)
print('Spreadsheet saved.')

#$ python3 write_posts.py
